# 딥페이크 탐지 모델 추론 - 제출용

이 노트북은 대회 제출용 추론 스크립트입니다.

## 평가 데이터 경로
- 입력: `./data/` (이미지 및 비디오 혼합)
- 출력: `./submission.csv`

## 출력 형식
- filename: 파일명 (확장자 포함)
- label: 0(Real) 또는 1(Fake)

## 1. 라이브러리 설치

In [1]:
# 필요한 라이브러리 설치 (CUDA 12.6 버전)
!pip install -q torch==2.7.1
!pip install -q torchvision==0.22.1
!pip install -q transformers==4.30.0
!pip install -q numpy==1.26.4
!pip install -q scipy==1.11.4
!pip install -q scikit-learn==1.3.2
!pip install -q opencv-python-headless==4.10.0.82
!pip install -q pandas
!pip install -q Pillow
!pip install -q tqdm
!pip install -q aifactory

print("✓ 라이브러리 설치 완료")

✓ 라이브러리 설치 완료


## 2. 라이브러리 Import

In [2]:
import warnings
warnings.filterwarnings("ignore")

import os
import glob
import cv2
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from PIL import Image as PILImage

import torch
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from transformers import ViTImageProcessor, ViTForImageClassification

print("✓ 라이브러리 임포트 완료")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Device: {torch.cuda.get_device_name(0)}")

✓ 라이브러리 임포트 완료
PyTorch version: 2.7.1+cu126
CUDA available: True
Device: NVIDIA GeForce RTX 4060


## 3. 모델 로드

In [3]:
# 모델 경로 설정
MODEL_PATH = "./model/deepfake_vit_final"

# 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 모델 및 프로세서 로드
print("\n모델 로딩 중...")
model = ViTForImageClassification.from_pretrained(MODEL_PATH)
processor = ViTImageProcessor.from_pretrained(MODEL_PATH)
model = model.to(device)
model.eval()

# 이미지 전처리 설정
image_mean = processor.image_mean
image_std = processor.image_std
image_size = processor.size["height"]

transform = Compose([
    Resize((image_size, image_size)),
    ToTensor(),
    Normalize(mean=image_mean, std=image_std)
])

print(f"✓ 모델 로드 완료")
print(f"  이미지 크기: {image_size}x{image_size}")
print(f"  라벨: {model.config.id2label}")

Using device: cuda

모델 로딩 중...
✓ 모델 로드 완료
  이미지 크기: 224x224
  라벨: {0: 'Real', 1: 'Fake'}


## 4. 추론 함수 정의

In [4]:
def predict_image(image_path):
    """
    이미지 파일에 대한 예측 수행
    
    Args:
        image_path: 이미지 파일 경로
    
    Returns:
        int: 예측 라벨 (0: Real, 1: Fake)
    """
    try:
        # 이미지 로드
        image = PILImage.open(image_path).convert('RGB')
        
        # 전처리
        pixel_values = transform(image).unsqueeze(0).to(device)
        
        # 추론
        with torch.no_grad():
            outputs = model(pixel_values=pixel_values)
            logits = outputs.logits
            predicted_label = logits.argmax(-1).item()
        
        return predicted_label
    
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        # 오류 발생 시 기본값 반환 (Real)
        return 0


def extract_video_frames(video_path, num_frames=5):
    """
    비디오에서 여러 프레임 추출
    
    Args:
        video_path: 비디오 파일 경로
        num_frames: 추출할 프레임 수
    
    Returns:
        list: PIL Image 리스트
    """
    frames = []
    
    try:
        cap = cv2.VideoCapture(str(video_path))
        if not cap.isOpened():
            return frames
        
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        if total_frames <= 0:
            cap.release()
            return frames
        
        # 균등하게 프레임 인덱스 선택
        if total_frames < num_frames:
            indices = list(range(total_frames))
        else:
            indices = np.linspace(0, total_frames - 1, num_frames, dtype=int)
        
        for idx in indices:
            cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
            ret, frame = cap.read()
            
            if ret and frame is not None:
                # BGR -> RGB 변환
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                pil_image = PILImage.fromarray(frame_rgb)
                frames.append(pil_image)
        
        cap.release()
    
    except Exception as e:
        print(f"Error extracting frames from {video_path}: {e}")
    
    return frames


def predict_video(video_path, num_frames=5):
    """
    비디오 파일에 대한 예측 수행 (여러 프레임 평균)
    
    Args:
        video_path: 비디오 파일 경로
        num_frames: 추출할 프레임 수
    
    Returns:
        int: 예측 라벨 (0: Real, 1: Fake)
    """
    try:
        # 프레임 추출
        frames = extract_video_frames(video_path, num_frames)
        
        if len(frames) == 0:
            # 프레임 추출 실패 시 기본값 반환
            return 0
        
        # 각 프레임에 대해 예측
        predictions = []
        
        for frame in frames:
            # 전처리
            pixel_values = transform(frame).unsqueeze(0).to(device)
            
            # 추론
            with torch.no_grad():
                outputs = model(pixel_values=pixel_values)
                logits = outputs.logits
                # Softmax로 확률 변환
                probs = torch.softmax(logits, dim=-1)
                # Fake 확률
                fake_prob = probs[0, 1].item()
                predictions.append(fake_prob)
        
        # 평균 확률로 최종 예측
        avg_fake_prob = np.mean(predictions)
        final_label = 1 if avg_fake_prob > 0.5 else 0
        
        return final_label
    
    except Exception as e:
        print(f"Error processing {video_path}: {e}")
        # 오류 발생 시 기본값 반환
        return 0


print("✓ 추론 함수 정의 완료")

✓ 추론 함수 정의 완료


## 5. 데이터 로드 및 추론

In [5]:
import os

# 데이터 경로 설정 (추론 환경과 개발 환경 모두 대응)
DATA_PATH = './data/'

# 개발 환경: data 디렉토리가 없으면 생성하거나 대체 경로 사용
if not os.path.exists(DATA_PATH):
    print(f"⚠️  '{DATA_PATH}' 디렉토리가 없습니다.")
    
    # 개발용 데이터 경로 (필요시 수정)
    DEV_DATA_PATH = '/home/lee/Final_project/데이터_원본(Data set)/'
    
    if os.path.exists(DEV_DATA_PATH):
        DATA_PATH = DEV_DATA_PATH
        print(f"✓ 개발 환경 데이터 경로 사용: {DATA_PATH}")
    else:
        # data 디렉토리 생성
        os.makedirs(DATA_PATH, exist_ok=True)
        print(f"✓ '{DATA_PATH}' 디렉토리 생성")
        print("⚠️  테스트용 데이터를 ./data/ 에 넣어주세요.")
else:
    print(f"✓ 데이터 경로 확인: {DATA_PATH}")

# 지원 파일 확장자
IMAGE_EXTS = ('.jpg', '.jpeg', '.png', '.bmp', '.webp')
VIDEO_EXTS = ('.mp4', '.avi', '.mov', '.mkv', '.webm')

# 파일 목록 수집
print("\n=== 파일 스캔 시작 ===")
all_files = []

if os.path.exists(DATA_PATH):
    for file in os.listdir(DATA_PATH):
        file_path = os.path.join(DATA_PATH, file)
        if os.path.isfile(file_path):
            all_files.append(file)

print(f"총 파일 수: {len(all_files)}")

# 파일 타입별 분류
image_files = [f for f in all_files if f.lower().endswith(IMAGE_EXTS)]
video_files = [f for f in all_files if f.lower().endswith(VIDEO_EXTS)]

print(f"  이미지: {len(image_files)}개")
print(f"  비디오: {len(video_files)}개")

✓ 데이터 경로 확인: ./data/

=== 파일 스캔 시작 ===
총 파일 수: 0
  이미지: 0개
  비디오: 0개


In [6]:
# 추론 수행
print("\n=== 추론 시작 ===")
results = []

# 이미지 추론
print("\n이미지 추론 중...")
for filename in tqdm(image_files, desc="Images"):
    file_path = os.path.join(DATA_PATH, filename)
    label = predict_image(file_path)
    results.append({
        'filename': filename,
        'label': label
    })

# 비디오 추론
print("\n비디오 추론 중...")
for filename in tqdm(video_files, desc="Videos"):
    file_path = os.path.join(DATA_PATH, filename)
    label = predict_video(file_path, num_frames=5)
    results.append({
        'filename': filename,
        'label': label
    })

print(f"\n✓ 추론 완료: {len(results)}개 파일")


=== 추론 시작 ===

이미지 추론 중...


Images: 0it [00:00, ?it/s]



비디오 추론 중...


Videos: 0it [00:00, ?it/s]


✓ 추론 완료: 0개 파일


## 6. 결과 저장

In [7]:
# 결과 확인
print(f"\n=== 추론 결과 확인 ===")
print(f"results 리스트 길이: {len(results)}")

if len(results) == 0:
    print("⚠️  추론 결과가 없습니다!")
    print("이전 셀(cell-11)을 먼저 실행해주세요.")
    # 빈 DataFrame 생성
    submission_df = pd.DataFrame(columns=['filename', 'label'])
else:
    # DataFrame 생성
    submission_df = pd.DataFrame(results)
    
    # 결과 확인
    print("\n=== 예측 결과 ===")
    print(submission_df.head(10))
    print(f"\n라벨 분포:")
    print(submission_df['label'].value_counts().sort_index())
    print(f"  Real (0): {(submission_df['label']==0).sum()}개")
    print(f"  Fake (1): {(submission_df['label']==1).sum()}개")
    
    # CSV 저장
    OUTPUT_PATH = "./submission.csv"
    submission_df.to_csv(OUTPUT_PATH, index=False)
    
    print(f"\n✓ 결과 저장 완료: {OUTPUT_PATH}")
    print(f"  총 {len(submission_df)}개 예측")


=== 추론 결과 확인 ===
results 리스트 길이: 0
⚠️  추론 결과가 없습니다!
이전 셀(cell-11)을 먼저 실행해주세요.


## 7. 모델 제출

In [8]:
# AIFactory 제출
import aifactory.score as aif

# 제출 키 설정 (마이페이지에서 발급)
COMPETITION_KEY = "7ad25b19-4651-4fc9-b7b3-126ca1f23876"

# 제출
aif.submit(
    model_name="deepfake_vit_baseline",
    key=COMPETITION_KEY
)

print("\n✓ 제출 완료!")

file : task
jupyter notebook
제출 완료

✓ 제출 완료!
